In [15]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# 鍵とnonceを生成
key = get_random_bytes(32)   # AES-256用
nonce = get_random_bytes(12) # GCM推奨サイズ
out={"original":[],"encrypted":[]}
# 暗号化
for i in [60,70,80,90]:
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    aad = b"header-data"  # 認証のみ対象にする追加データ
    cipher.update(aad)
    original="secret message"*i
    plaintext = (original).encode('utf-8')
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)

    # 暗号化結果をまとめる（Nonce + Ciphertext + Tag）
    encrypted_package = nonce + ciphertext + tag
    print("暗号化パッケージ:", encrypted_package)

    # 復号化
    nonce_dec = encrypted_package[:12]
    ciphertext_dec = encrypted_package[12:-16]
    tag_dec = encrypted_package[-16:]

    cipher_dec = AES.new(key, AES.MODE_GCM, nonce=nonce_dec)
    cipher_dec.update(aad)
    decrypted = cipher_dec.decrypt_and_verify(ciphertext_dec, tag_dec)

    print("復号結果:", decrypted)
    import base64
    encoded_str = base64.b64encode(encrypted_package).decode('utf-8')
    print("Base64文字列:", encoded_str)
    print(len(encoded_str))
    print(len(original))
    out["original"].append(len(original))
    out["encrypted"].append(len(encoded_str))
    

暗号化パッケージ: b'\xdf\xa1u\x85\xd0\x97\xb8o\x985\xb9\xb7y\xf1\xc8\x88\x7f\x9f:jaYA\xe9\xf3\x8f\x1b\x0b:\xe2\x99\x15f\x8b\x91\x9c\x0f\x85<-\xbf\x84\x91\x85\xd2\x0f#\x06*\xc42Z\xc4\x8b\x9bW`A\xf9\xfc\xa4\x8f\x16n/\xdbP\x98\xa7\xfd\x9c\xfe%\xb5\x05\x83q\xa7B/jl\x9c\xec\xddsV\x8e\x04S\xd5=\xf7\x9a\xeek\x8f\x8a\xa5\xc5\x8e\xb1\xc2\x0c\xfch\x1a\x03\x182\xd5\x02\xce\xac0\xc5\xb2\x9f\x87X\'\xe3\x89\xe7\x16?\x07\x17\xc7Ck\xffZQ\x1cK\xabWE\x05\xf7\'`\xd0\x94\xe7Zv\xda\xf9\xe8&\x9e\xed}\xd1\x81h\xfa\x85@\x0b\xac0\xf7Vb\x90\x828|d\x12w\xed\x96\xaa\xc2\xf8mL\xc9\x0f\xe8x\xdd\xce\x80\xe9U\xe7f\x99\x8e\xe1\x1a\x9d\x13\xfd\x06R\xcc4\'\xd3o\xd9\xb8\xbe\xc8K5\x7fm\x87\x1aw\xf3\xa5\xc7\x01\xe3\'F-q\xfa\xb2&\xc2\xde\xe1\xb0\x1c\xc7\xccU\xd5\x07\xd4]n\xbe\xba\xe4%\xde\x01\xad!\xa8\r\x9d\x0f-\xc6\xa0IA\xbf\x0f\x1ac\x88\xf4e\x99\xea 5\xfbI&\xfa\x13\x96\x97L\xba\xd7\x91^5R\xbb\x0c\xaf^#\xf4\xdc\x83z\x9a\t\xe6\xfe\xeb\xf3\xa1\x0e~\x1f\xc8\xde\xdc}\x95B\xa4R\xbb\x12N#\xc9t\xfd\x8d\xab0sH\x82\xe4\x1a\xf4\xb6mW\xffAk\

In [17]:
import pandas as pd

pd.DataFrame(out)


,original,encrypted
0,840,1160
1,980,1344
2,1120,1532
3,1260,1720
